In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, maximize, SolverFactory, RangeSet, Binary

In [11]:
# Initialize the Pyomo model
model = ConcreteModel()

# Time steps
T = 10
model.t = RangeSet(0, T-1)

# Nodes and edges definitions
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
edges = ['AB', 'AC', 'AF', 'BC', 'BD', 'BE', 'CE', 'CF', 'DE', 'EG', 'FB', 'FG']
capacities = {'AB': 3, 'AC': 6, 'AF': 1, 'BC': 3, 'BD': 10, 'BE': 4, 
              'CE': 4, 'CF': 4, 'DE': 5, 'EG': 5, 'FB': 12, 'FG': 7}

# Maximum number of active pipes
k = 10

# Decision variables for water flow on each edge at each time
model.X = Var(edges, model.t, within=NonNegativeReals)

# Decision variables for water release at each node at each time
model.Y = Var(nodes, model.t, within=NonNegativeReals)

# Binary decision variables for activation status of pipes
model.Z = Var(edges, within=Binary)

# Flow entering function at node A
def f(t):
    if t >= 0:
        return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)
    else:
        return 0

# Objective: Maximize the total water flow to point G over all time steps
model.obj = Objective(expr=sum(model.X['EG', t] + model.X['FG', t] for t in model.t), sense=maximize)

# Constraint: Flow conservation and capacity constraints
def flow_conservation_rule(model, e, t):
    return model.X[e, t] <= model.Z[e] * capacities[e]

model.flow_conservation = Constraint(edges, model.t, rule=flow_conservation_rule)

# Flow entering and leaving each node
def node_balance_rule(model, n, t):
    outflow = sum(model.X[e, t] for e in edges if e[0] == n) + model.Y[n, t]
    if t == 0:
        return outflow == 0

    inflow = sum(model.X[e, t - 1] for e in edges if e[1] == n)

    if n == 'A':
        return outflow == f(t - 1)
    
    return inflow == outflow

model.node_balance = Constraint(nodes, model.t, rule=node_balance_rule)

# Activation constraints for pipes
model.active_pipes = Constraint(expr=sum(model.Z[e] for e in edges) <= k)

# Solve the model
solver = SolverFactory('gurobi')
result = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
print()
for e in edges:
    for t in model.t:
        if model.X[e, t].value > 0:
            print(f"Flow in {e} at time {t}: {model.X[e, t].value}")

print()
for n in nodes:
    for t in model.t:
        print(f"Drainage at node {n} at time {t}: {model.Y[n, t].value}")



print()

for e in edges:
    print(f"Activation staus of edge {e}: {model.Z[e].value}")


Objective value: 49.12266666666667

Flow in AB at time 1: 2.25
Flow in AB at time 3: 3.0
Flow in AB at time 4: 3.0
Flow in AB at time 5: 3.0
Flow in AB at time 6: 3.0
Flow in AB at time 7: 2.7959999999999994
Flow in AC at time 2: 2.6326666666666667
Flow in AC at time 3: 2.4653333333333336
Flow in AC at time 4: 4.698
Flow in AC at time 5: 6.0
Flow in AC at time 6: 6.0
Flow in AC at time 7: 6.0
Flow in AF at time 1: 0.75
Flow in AF at time 2: 1.0
Flow in AF at time 5: 0.5306666666666668
Flow in AF at time 6: 1.0
Flow in AF at time 8: 1.0
Flow in BE at time 2: 2.25
Flow in BE at time 4: 3.0
Flow in BE at time 5: 3.0
Flow in BE at time 6: 3.0
Flow in BE at time 7: 3.0
Flow in BE at time 8: 2.7959999999999994
Flow in CE at time 5: 2.0
Flow in CE at time 6: 2.0
Flow in CE at time 7: 2.0
Flow in CE at time 8: 2.0
Flow in CF at time 3: 2.6326666666666667
Flow in CF at time 4: 2.4653333333333336
Flow in CF at time 5: 2.6980000000000004
Flow in CF at time 6: 4.0
Flow in CF at time 7: 4.0
Flow in